# GRPO 

GRPO = Group Relative Policy Optimization – a critic-free, PPO-style reinforcement-learning algorithm invented by the DeepSeek team to cheaply push LLMs’ reasoning skills. It trains the model by comparing several candidate answers within the same prompt and nudging the policy toward the best-scoring ones, using the group’s average as the baseline. No value-network, far less VRAM, and surprisingly stable updates even with tiny datasets. 
Let' focus on its key features.

GRPO’s novelty lies in reframing the policy optimization problem to be more resource-efficient and naturally aligned with comparative rewards. By using multiple sampled outcomes as its own baseline, it removes the need for training a large critic network and capitalizes on the strengths of preference-based reward models. This led to roughly 50% reduction in memory cost for RLHF-style training and enabled new successes in aligning models with complex reasoning tasks. It’s a prime example of how a relatively simple algorithmic change (using group relative advantages) can have outsized practical impact in large-scale RL.

**Critic-based RL (PPO, A2C, RLHF)**: 
In PPO-based RL fine-tuning of language models, the value model is typically an identical copy of the language model (with an extra value head) that must be trained in tandem with the policy, doubling the number of large models one must backpropagate through.

- Critic network: predicts the expected reward for each state, prompt, or context.
- Policy network: generates actions (tokens, sentences, moves) meant to maximise reward.
- Advantage: after an action, compute advantage = real_reward – critic_estimate.
- Policy update: scale the log-prob gradient by the advantage—positive ⇒ reinforce, negative ⇒ suppress.
- Critic update: regress its prediction toward the observed reward (MSE loss).
- Result: the learned baseline cuts gradient variance, so training is more stable and sample-efficient while steering the policy toward higher long-term reward.


**“Critic-free” RL (GRPO)** 
The most prominent difference of GRPO is the removal of the separate value (critic) model.  GRPO does away with this, which cuts the memory and compute requirements roughly in half for the same model size. By avoiding the critic, GRPO streamlines the RL pipeline, reducing the “moving parts” that need careful tuning. This makes the optimization problem simpler: fewer gradients to stabilize and no need to ensure the value function is calibrated correctly. As Greg (2025) notes, “PPO has 4 LLMs in the mix... Not only is this expensive, there are many moving parts. GRPO helps simplify things.

- RL loop throws away the extra neural network that normally learns a value function (the “critic”) and just updates the policy directly with a cleverly chosen baseline.
- No critic network: the value-function head is dropped entirely.
- Policy only: for each prompt, sample K candidate answers and score each with an automatic reward signal.
- Baseline: take the average reward across those K samples as the reference point.
- Advantage: an answer’s reward minus that average decides whether to reinforce (positive) or suppress (negative) its logits.
- Update rule: run the usual PPO-style clipped loss, scaling gradients by this advantage instead of a learned critic.


- **On-the-fly Baseline from Group** – Natural Use of Comparisons: By using the group’s own average performance as a baseline, GRPO leverages the relative feedback that is often inherent in preference-based rewards. Reward models in RLHF are usually trained on pairwise comparisons (“output A is better than output B”), which means the reward score is only meaningful up to an arbitrary scale or offset. GRPO’s relative advantage aligns well with this – it cares only that one output is better than another for the same query, not the absolute value of the reward.

This built-in normalization (subtracting the mean) helps stabilize training, as it automatically recenters the advantage to zero for an average output, and the optional normalization by standard deviation scales the update signals to a reasonable range.

In traditional actor-critic methods, one has to carefully tune value function learning and advantage normalization (e.g. using Generalized Advantage Estimation and value clipping) to get similar effects – GRPO achieves a form of this implicitly through sampling. In short, GRPO’s “group relative” approach turns the policy itself into its own baseline estimator, which is a novel twist on variance reduction in policy gradients.

- **Fewer Hyperparameters and Modes of Failure**: Removing the critic eliminates hyperparameters like the value loss weight or the need to balance value vs. policy learning speed. It also removes failure modes where the value function lagged or diverged, which in PPO can destabilize training if the critic doesn’t accurately predict returns. GRPO only needs tuning of the group size $G$ (how many samples to draw) and retains the usual PPO hyperparameters like clip ratio $\varepsilon$ and KL penalty $\beta$. The training is more straightforward to monitor since one doesn’t need to watch for value function loss anomalies. The DeepSeek authors note that GRPO significantly reduced the engineering complexity of their RL stage. In practice, they found it “significantly reducing training resources” and yet still yielding strong performance gains. [Arxiv - Deepseek Math Paper](https://arxiv.org/pdf/2402.03300#:~:text=Furthermore%2C%20we%20introduce%20the%20Group,Instruct%2C%20including%20both)

- **Scalable to Programmatic Rewards (Less Human Feedback)**: GRPO has been highlighted as particularly useful for tasks with verifiable or programmatic outcomes. 

If you can write a script or have a ground-truth way to score an answer (as in math problems with known solutions, code tasks with test cases, games with clear rules), GRPO allows you to improve an agent *without needing large human preference datasets*. This is a more scalable alternative to traditional RLHF which requires many human comparisons or a well-trained reward model for subjective tasks. DeepSeek’s projects took advantage of this by using rule-based reward functions for mathematics (checking numeric answers) and coding (running submitted code against tests).
They avoided training a learned reward model for those tasks, citing concerns about reward model drift and hacking over long training runs. GRPO thus demonstrated that reinforcement fine-tuning (RFT) could be done with almost no human labels, given a suitable automated reward – a compelling proposition for domains like math, programming, or any scenario where outcomes can be automatically checked for correctness. Recent courses and engineering blogs have started to promote GRPO for this reason, teaching how to design such reward functions and use GRPO to “steer models toward high-quality behavior” on multi-step reasoning tasks.
 





